# Calcualte performance rankings from cleaned results

In [1]:
# read cleaned results
import pandas as pd
from pathlib import Path
import pdb, os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

agg_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results_with_default.csv"))
agg_df = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results.csv"))

fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))
fold_df = pd.read_csv(Path("./cleaned_results/tuned_fold_results.csv"))

# make sure output folder exists
output_folder = Path("./performance_rankings")
output_folder.mkdir(exist_ok=True)  

/tmp/ipykernel_4081450/4270952197.py:11: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))


In [2]:
import numpy as np

def get_rank_table(df, metric, table_idx=4):
    """
    NOTE: the df needs to have the following columns defined:
    - {metric}_rank_mean
    - normalized_{metric}__test_mean
    """
    if table_idx == 1:
        from analysis.table1 import DATASETS
    elif table_idx == 2:
        from analysis.table2 import DATASETS
    elif table_idx == 4:
        from analysis.table4 import DATASETS
        
    df = df[df["dataset_name"].isin(DATASETS)]

    overall_ranks = df.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "median", "count"],
            f"normalized_{metric}__test_mean": ["mean", "median"],
            f"normalized_{metric}__test_std": ["mean", "median"],
            f"train_per_1000_inst_mean_{metric}": ["mean", "median"],        
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    # overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].apply(lambda x: "{:d}".format(int(x)))

    # mean/median mean-rank
    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"{metric}_rank_mean", "median")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "median")].apply(lambda x: "{:d}".format(int(x)) if int(x) == x else "{:.1f}".format(x))
    
    # normalized metric - mean and std over folds
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "median")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"normalized_{metric}__test_std", "median")] = overall_ranks.loc[:,(f"normalized_{metric}__test_std", "median")].apply(lambda x: "{:.2f}".format(x))


    # normalized runtime
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "mean")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "mean")].apply(lambda x: "{:.2f}".format(x))
    overall_ranks.loc[:, (f"train_per_1000_inst_mean_{metric}", "median")] = overall_ranks.loc[:,(f"train_per_1000_inst_mean_{metric}", "median")].apply(lambda x: "{:.2f}".format(x))
   


    final_table = overall_ranks.set_index("alg_name")

    return final_table


### Save rank tables to file

In [3]:
# save rank tables to csv and latex
from analysis_utils import ALG_DISPLAY_NAMES
table_idx = 1

metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

final_tables = {}

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    agg_df_with_default

    # first with default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df_with_default, metric, table_idx = table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank_with_default.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank_with_default.tex", index=True, escape=False)

    # now without default hparams as its own alg
    final_tables[metric] = get_rank_table(agg_df, metric, table_idx=table_idx)

    # save to csv, latex
    final_tables[metric].to_csv(output_folder / f"{metric}_rank.csv", index=True)
    final_tables[metric].to_latex(output_folder / f"{metric}_rank.tex", index=True, escape=False)

/tmp/ipykernel_4081450/2689206630.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '3' '1' '1' '1' '10']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
/tmp/ipykernel_4081450/2689206630.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['38' '38' '35' '37' '40' '41' '43' '41' '42' '42' '42' '35' '41' '43'
 '42' '43' '44' '39' '42' '38' '41' '39' '44' '41' '44' '44' '42' '42'
 '42' '45' '43' '39' '45' '43' '43' '44' '44' '45' '45' '45' '43' '45'
 '43' '45' '45']' has dtype incompatible with

/tmp/ipykernel_4081450/2689206630.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '2' '1']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].apply(lambda x: "{:d}".format(int(x)))
/tmp/ipykernel_4081450/2689206630.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['21' '20' '21' '22' '22' '22' '19' '22' '20' '23' '21' '21' '22' '23'
 '22' '23' '22' '22' '23' '23' '23' '23' '23']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].apply(lambda x: "{:d}".format(

In [4]:
final_tables['Accuracy']

Accuracy_rank_mean                    \
                              min max   mean median   
alg_name                                              
TabPFN                          1  21   5.66    3.5   
CatBoost                        1  20   5.91      4   
XGBoost                         1  21   7.57      6   
ResNet                          1  22   8.26    8.5   
SAINT                           1  22   8.56      7   
NODE                            1  22   8.60      8   
FTTransformer                   1  19   8.74    8.5   
Ours                            1  22   8.77    6.5   
RandomForest                    1  20   8.93      8   
LightGBM                        1  23   9.32    8.5   
SVM                             1  21   9.98   10.5   
MLP-rtdl                        1  21  10.46   10.5   
DANet                           1  22  10.77     10   
DeepFM                          1  23  11.52     12   
DecisionTree                    1  22  12.85     14   
STG                             1  23  13.01     14   
MLP                             1  22  13.06     14   
LinearModel                     1  22  13.21   15.5   
TabTransformer                  1  23  14.03     15   
TabNet                          1  23  14.04     15   
KNN                             1  23  15.14     16   
VIME                            2  23  16.42     18   
NAM                             1  23  17.41     19   

               normalized_Accuracy__test_mean         \
                                         mean median   
alg_name                                               
TabPFN                                   0.85   0.94   
CatBoost                                 0.88   0.95   
XGBoost                                  0.83   0.92   
ResNet                                   0.79   0.88   
SAINT                                    0.77   0.91   
NODE                                     0.78   0.86   
FTTransformer                            0.80   0.86   
Ours                                     0.73   0.90   
RandomForest                             0.80   0.87   
LightGBM                                 0.79   0.85   
SVM                                      0.74   0.85   
MLP-rtdl                                 0.70   0.79   
DANet                                    0.78   0.83   
DeepFM                                   0.69   0.79   
DecisionTree                             0.68   0.77   
STG                                      0.64   0.74   
MLP                                      0.64   0.64   
LinearModel                              0.59   0.67   
TabTransformer                           0.57   0.73   
TabNet                                   0.60   0.69   
KNN                                      0.54   0.60   
VIME                                     0.45   0.43   
NAM                                      0.42   0.40   

               normalized_Accuracy__test_std         \
                                        mean median   
alg_name                                              
TabPFN                                  0.32   0.24   
CatBoost                                0.25   0.14   
XGBoost                                 0.29   0.16   
ResNet                                  0.26   0.16   
SAINT                                   0.26   0.19   
NODE                                    0.22   0.15   
FTTransformer                           0.27   0.17   
Ours                                    0.26   0.18   
RandomForest                            0.27   0.19   
LightGBM                                0.31   0.17   
SVM                                     0.22   0.16   
MLP-rtdl                                0.24   0.14   
DANet                                   0.27   0.18   
DeepFM                                  0.25   0.21   
DecisionTree                            0.31   0.20   
STG                                     0.25   0.14   
MLP                                     0.25   0.15   
LinearModel                